In [60]:
from bs4 import BeautifulSoup
import requests
import re

# Web scraping from the radar AvroTros website
search_link = "https://radar.avrotros.nl/forum/energie-f51/index-s"

forum_url = BeautifulSoup(requests.get(search_link).content, 'html.parser')

page_index = 0

dict_topics = {}

for i in range(0, 100, 25):
    page_index = i

    forum_url = BeautifulSoup(requests.get(search_link + str(page_index) + '.html').content, 'html.parser')

    items = forum_url.find_all(class_=lambda x: x and x.startswith("row bg"))

    for item in items:
        title = item.find('a', class_='topictitle')
        topic_title = title.text

        topic_link = "https://radar.avrotros.nl/forum" + "/" + title['href'].lstrip('./..').split('.html')[0] + '.html'

        date_element = item.find("div", class_="topic-poster responsive-hide left-box").find("time")
        date = date_element.text

        code_link = topic_link[-11:]
        code = code_link[:6]

        reactions_element = item.find("dd", class_="posts")
        reactions = reactions_element.text
        # Remove the 'Reacties' text
        reactions = reactions[:-9]
        reactions = int(reactions)

        dict_topics[topic_title] = topic_link, date, code, reactions

# Convert dict to dataframe
import pandas as pd

df_topics = pd.DataFrame.from_dict(dict_topics, orient='index', columns=['link', 'date', 'code', 'reactions'])

df_topics

,link,date,code,reactions
De energiemarkt en salderen: hoe komt de prijsvorming tot stand?,https://radar.avrotros.nl/forum/energie-f51/de...,22 okt 2021 14:18,197725,48
"Woningen met verkeerde energielabels, wie heeft daar ervaring mee?",https://radar.avrotros.nl/forum/energie-f51/wo...,13 feb 2023 19:48,200283,15
Energieleverancier kiezen met zonnepanelen,https://radar.avrotros.nl/forum/energie-f51/en...,05 jan 2021 16:57,195768,100
Hogere tarieven Vattenfall ontlopen,https://radar.avrotros.nl/forum/energie-f51/ho...,23 dec 2022 16:42,199987,9
"Essent - energiebelasting teruggave/aanmerking woning als ""woonverblijf""",https://radar.avrotros.nl/forum/energie-f51/es...,10 feb 2023 09:38,200251,5
...,...,...,...,...
Vattenfall wil Warmtelink plaatsen voor digitaal uitlezen verbruik,https://radar.avrotros.nl/forum/energie-f51/va...,02 jun 2021 17:18,196961,47
Energierekening opsplitsen in Netbeheer en Levering,https://radar.avrotros.nl/forum/energie-f51/en...,03 mar 2022 09:41,198540,17
Faillisement Energie Welkom: wat kan / mag ik?,https://radar.avrotros.nl/forum/energie-f51/fa...,15 nov 2021 09:41,197882,31
"Na failliete Welkom Energie nu de rekening van Eneco voor november, wat nu?",https://radar.avrotros.nl/forum/energie-f51/na...,17 dec 2021 13:29,198056,176


In [71]:
# Get first ten rows within df
topics = df_topics.head(2)


dict_messages = {}
message_id = 1

for topic, row in topics.iterrows():
    page_index = 0
    while True:
        if page_index > row['reactions']:
            break
        page_link = "https://radar.avrotros.nl/forum/viewtopic.php?t=" + row['code'] + "&start=" + str(page_index)
        print(page_link)
        page = requests.get(page_link)
        if page.status_code != 200:
            break
        soup = BeautifulSoup(page.content, 'html.parser')
        posts = soup.find_all('div', class_=lambda x: x and x.startswith("post has-profile bg"))
        for post in posts:
            content = post.find("div", class_="content")
            # Remove blockquotes to prevent double counting
            for blockquote in content.find_all("blockquote"):
                blockquote.decompose()
            content = content.text

            date = post.find('p', class_="author").find('time').text

            dict_messages[message_id] = topic, content, date
            message_id += 1

        page_index += 20


df_messages = pd.DataFrame.from_dict(dict_messages, orient='index', columns=['topic', 'content', 'date'])

# Removing newlines and tabs from content cells.
df_messages['content'] = df_messages['content'].str.replace('\n', '')
df_messages['content'] = df_messages['content'].str.replace('\t', '')

# Convert dates to datetime
from datetime import datetime
import locale
locale.setlocale(locale.LC_ALL, "nl_NL")

df_messages["date"] = df_messages["date"].str.split(" ").apply(lambda x: " ".join([i.capitalize() for i in x]))

print(datetime.strptime("22 Okt 2021 14:32", "%d %b %Y %H:%M"))

df_messages['date'] = df_messages['date'].apply(lambda x: datetime.strptime(x, "%d %b %Y %H:%M"))

df_messages



https://radar.avrotros.nl/forum/viewtopic.php?t=197725&start=0
https://radar.avrotros.nl/forum/viewtopic.php?t=197725&start=20
https://radar.avrotros.nl/forum/viewtopic.php?t=197725&start=40
https://radar.avrotros.nl/forum/viewtopic.php?t=200283&start=0
2021-10-22 14:32:00


,topic,content,date
1,De energiemarkt en salderen: hoe komt de prijs...,"Goedendag,In navolging van het inmiddels geslo...",2021-10-22 14:18:00
2,De energiemarkt en salderen: hoe komt de prijs...,3.U verbruikt en produceert stroomDezelfde pri...,2021-10-22 14:52:00
3,De energiemarkt en salderen: hoe komt de prijs...,Heel leerzaam artikel.,2021-10-22 17:23:00
4,De energiemarkt en salderen: hoe komt de prijs...,Dank voor de uitgebreide uitleg.,2021-10-22 17:57:00
5,De energiemarkt en salderen: hoe komt de prijs...,Ik heb de mods gevraagd dit topic sticky te ma...,2021-10-22 18:11:00
...,...,...,...
61,"Woningen met verkeerde energielabels, wie heef...",Helaas blijkt maar weer wat in theorie de perf...,2023-02-14 09:42:00
62,"Woningen met verkeerde energielabels, wie heef...",Jouw jaren '90 woning is sowieso al veel beter...,2023-02-14 09:46:00
63,"Woningen met verkeerde energielabels, wie heef...",Volgens mij had ik het over dat label. Niet we...,2023-02-14 10:35:00
64,"Woningen met verkeerde energielabels, wie heef...",Wat voor verwarming heb je? Vanaf half jaren 9...,2023-02-14 11:05:00
